In [109]:
import pandas as pd
from sqlalchemy import create_engine

In [110]:
csv_file = "Resources/netflix_titles.csv"
netflix_data = pd.read_csv(csv_file)
netflix_data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [111]:
import requests
import json
from pprint import pprint

In [112]:
url = "http://www.omdbapi.com/?t="
api_key = "&apikey=77d3b433"

In [113]:
netflix_data = netflix_data[netflix_data["type"] == "Movie"]
movies = netflix_data["title"]
movies = movies.head(30)
movies


0               Norm of the North: King Sized Adventure
1                            Jandino: Whatever it Takes
4                                          #realityhigh
6                                              Automata
7                    Fabrizio Copano: Solo pienso en mi
9                                           Good People
10                          Joaquín Reyes: Una y no más
11                              Kidnapping Mr. Heineken
12                             Krish Trish and Baltiboy
13             Krish Trish and Baltiboy: Battle of Wits
14       Krish Trish and Baltiboy: Best Friends Forever
15            Krish Trish and Baltiboy: Comics of India
16    Krish Trish and Baltiboy: Oversmartness Never ...
17                    Krish Trish and Baltiboy: Part II
18         Krish Trish and Baltiboy: The Greatest Trick
19                                                 Love
20                                    Manhattan Romance
21                                          Moon

In [117]:
#Run a for loop to acquire data from OMDB API
imdb_rating = []
meta_rating = []
awards = []
not_found = 0
found_movies = []

for movie in movies:
    response = requests.get (url + movie + api_key)
    omdb = response.json()
    try:
        imdb_rating.append(omdb["imdbRating"])
        meta_rating.append(omdb["Metascore"])
        awards.append(omdb["Awards"])
        found_movies.append(movie)
    except:
        not_found = not_found + 1
        pass
        

In [118]:
#Check how many movies were not found
not_found

9

In [140]:
#Create dataframe from information acquired from API
omdb_df = pd.DataFrame(list(zip(found_movies, imdb_rating, meta_rating, awards)),columns = ["Movie_Title", "IMDB_Rating", "Metacritic_Rating", "Awards"])
omdb_df = omdb_df[omdb_df!="N/A"].dropna()
omdb_df.reset_index()

,index,Movie_Title,IMDB_Rating,Metacritic_Rating,Awards
0,2,Automata,6.1,37,6 nominations.
1,7,Love,6.1,51,2 wins & 1 nomination.
2,10,Rolling Papers,6.2,60,1 win.
3,11,Stonehearst Asylum,6.8,52,1 win & 1 nomination.
4,13,6 Years,5.6,53,4 nominations.
5,17,Sierra Burgess Is A Loser,5.8,60,2 nominations.


In [ ]:
rds_connection_string = "postgres:postgres@localhost:5432/netflix_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
engine.table_names()

In [ ]:
omdb_df.to_sql(name='Netflix_Ratings', con = engine, if_exists = 'append', index=True)